In [1]:
import tensorflow as tf

import numpy as np

In [2]:
data = np.load('data.npy', allow_pickle=True)

train = data[:54000]
test = data[54000:]

train_X = []
train_y = []
for x in train:
    train_X.append(x[0])
    train_y.append(x[1])

test_X = []
test_y = []
for x in test:
    test_X.append(x[0])
    test_y.append(x[1])

train_X = np.array(train_X)
train_y = np.array(train_y)

test_X = np.array(test_X)
test_y = np.array(test_y)

In [4]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

epochs=25
batch_size=32

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(train_X.shape[1], train_X.shape[2], train_X.shape[3])),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(78, activation='softmax')
])

model.summary()

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 78)             │        40,014 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,607,950 (21.39 MB)

 Trainable params: 5,607,950 (21.39 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath="250epochs_conv.keras",
                               save_best_only=True,
                               verbose=0)

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, 
                    validation_data=(test_X, test_y), callbacks=[cp]).history

Epoch 1/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1045s 618ms/step - accuracy: 0.4280 - loss: 2.3942 - val_accuracy: 0.9938 - val_loss: 0.0219
Epoch 2/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1037s 614ms/step - accuracy: 0.9904 - loss: 0.0435 - val_accuracy: 0.9998 - val_loss: 3.4698e-04
Epoch 3/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1035s 613ms/step - accuracy: 0.9945 - loss: 0.0236 - val_accuracy: 0.9971 - val_loss: 0.0117
Epoch 4/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1037s 614ms/step - accuracy: 0.9957 - loss: 0.0286 - val_accuracy: 0.9980 - val_loss: 0.0146
Epoch 5/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1033s 612ms/step - accuracy: 0.9964 - loss: 0.0236 - val_accuracy: 0.9996 - val_loss: 0.0014
Epoch 6/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1032s 612ms/step - accuracy: 0.9955 - loss: 0.0344 - val_accuracy: 0.9993 - val_loss: 0.0019
Epoch 7/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1030s 610ms/step - accuracy: 0.9961 - loss: 0.0272 - val_accuracy: 0.9969 - val_loss: 0.0235
Epoch 8/25
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 1034s 612